<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/GRPO_Study_Ntbk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRPO Study Notebook

Generate Mulptiple responses from the model for the reward model training

In [ ]:
# Load GPT model & tokenizer
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Define a question (prompt)
prompt = "What are the benefits of exercise?"

# Tokenize the input
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Generate multiple responses using sampling
num_responses = 5  # Generate 5 different completions
responses = model.generate(
    input_ids,
    max_length=50,
    do_sample=True,  # Enables sampling instead of greedy decoding
    top_k=50,  # Consider top 50 tokens at each step
    top_p=0.9,  # Nucleus sampling: keeps top tokens contributing to 90% probability
    temperature=0.7,  # Controls randomness (lower = more deterministic)
    num_return_sequences=num_responses  # Generates multiple responses
)

# Decode and print responses
decoded_responses = [tokenizer.decode(output, skip_special_tokens=True) for output in responses]
for i, response in enumerate(decoded_responses):
    print(f"Response {i+1}: {response}")

Example: Using a Reward Model to Score Responses
Assume we have a pretrained reward model that scores responses from 0 to 1, where:

1.0 = Excellent
0.0 = Poor

In [ ]:
class RewardModel(nn.Module):
    """Simple reward model that scores responses."""
    def __init__(self, embedding_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(embedding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1),  # Output: a single score per response
            nn.Sigmoid()  # Normalize score between 0 and 1
        )

    def forward(self, response_embedding):
        return self.fc(response_embedding)

# Instantiate reward model
reward_model = RewardModel(embedding_dim=768)  # Assuming GPT's embeddings

# Convert responses to embeddings
response_inputs = tokenizer(decoded_responses, return_tensors="pt", padding=True, truncation=True)
response_embeddings = model.transformer.wte(response_inputs["input_ids"]).mean(dim=1)  # Averaging token embeddings

# Score each response
reward_scores = reward_model(response_embeddings).squeeze()

# Print scores
for i, (response, score) in enumerate(zip(decoded_responses, reward_scores.tolist())):
    print(f"Response {i+1}: {response} | Score: {score:.3f}")


# Policy
Train the Policy with GRPO
Train the policy network to predict higher scores for better responses.
Update it using a reinforcement learning algorithm like GRPO.
Example: Training the Policy Network

In [ ]:
policy_optimizer = optim.Adam(policy_net.parameters(), lr=3e-4)

for batch in training_data:  # Assume training_data is a dataset of responses + rewards
    response_embeddings = model.transformer.wte(batch["response_ids"])

    predicted_scores = policy_net(response_embeddings.mean(dim=1))
    advantage = batch["true_rewards"] - predicted_scores.detach()  # Compute advantage

    policy_loss = -torch.min(
        predicted_scores * advantage,
        torch.clamp(predicted_scores, 0.8, 1.2) * advantage
    ).mean()

    policy_optimizer.zero_grad()
    policy_loss.backward()
    policy_optimizer.step()


Using the Policy for Action Selection
Once the policy is trained, it can be used to select actions in the environment.

Example: Action Selection in a Trained Policy

In [ ]:
import torch

def select_action(policy_net, state):
    """Selects an action based on the trained policy."""
    with torch.no_grad():  # No gradients needed for inference
        action_probs = policy_net(state)
        action = torch.multinomial(action_probs, 1)  # Sample from the policy distribution
    return action.item()

# Example usage:
state = torch.rand((1, 4))  # Example state (assuming 4D input)
action = select_action(policy_net, state)
print(f"Selected Action: {action}")


# Fine-Tuning an LLM with a Policy Network
Below is a simplified PyTorch example that fine-tunes an LLM (e.g., GPT) using a trained policy.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the LLM (e.g., GPT-2 for simplicity)
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Define a simple policy network that scores responses
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # Output: scalar score for each response
        )

    def forward(self, x):
        return self.fc(x)

# Instantiate the policy network
policy_net = PolicyNetwork(input_dim=768)  # GPT's embedding size
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Simulated training batch
texts = ["What is the capital of France?", "How does photosynthesis work?"]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

# Step 1: Generate responses from the LLM
with torch.no_grad():
    outputs = model.generate(inputs["input_ids"], max_length=50)

# Step 2: Get response embeddings (used as input to policy network)
response_embeddings = model.transformer.wte(outputs)  # Word token embeddings
response_scores = policy_net(response_embeddings.mean(dim=1))  # Score each response

# Step 3: Compute policy loss (GRPO-style clipped loss)
baseline = response_scores.mean()  # Baseline for advantage calculation
advantage = response_scores - baseline  # Compute advantage function
policy_loss = -torch.min(response_scores * advantage, torch.clamp(response_scores, 0.8, 1.2) * advantage).mean()

# Step 4: Backpropagate the loss to fine-tune the LLM
optimizer.zero_grad()
policy_loss.backward()
optimizer.step()

print(f"Policy Loss: {policy_loss.item()}")
